In [1]:
import pyspark as ps
import pyspark.sql.functions as f
from pyspark import SQLContext
from pyspark.sql.types import IntegerType, DateType, TimestampType
from datetime import datetime

spark = (ps.sql.SparkSession.builder 
        .master("local[4]") 
        .appName("sparkSQL exercise") 
        .getOrCreate()
        )
sc = spark.sparkContext

In [2]:
sc

<SparkContext master=local[4] appName=sparkSQL exercise>

In [3]:
sqlContext = SQLContext(sc)

In [4]:
df = sqlContext.read.csv("uk100.csv", header=True)

In [5]:
df.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- rank: string (nullable = true)
 |-- last_week_rank: string (nullable = true)
 |-- hmm: string (nullable = true)
 |-- title: string (nullable = true)
 |-- artist: string (nullable = true)
 |-- label: string (nullable = true)
 |-- peak_rank: string (nullable = true)
 |-- weeks_on_chart: string (nullable = true)
 |-- week_of: string (nullable = true)



In [6]:
df.createOrReplaceTempView('test')

In [7]:
result = spark.sql('''SELECT *
            FROM test
            LIMIT 30
            ''')
result.show()

+---+----+--------------+--------+--------------------+--------------------+--------------------+---------+--------------+----------------+
|_c0|rank|last_week_rank|     hmm|               title|              artist|               label|peak_rank|weeks_on_chart|         week_of|
+---+----+--------------+--------+--------------------+--------------------+--------------------+---------+--------------+----------------+
|  0|   1|            11|        |               RIVER|      ELLIE GOULDING|             POLYDOR|        1|             5|December 27 2019|
|  1|   2|             8|        |ALL I WANT FOR CH...|        MARIAH CAREY|            COLUMBIA|        2|            99|December 27 2019|
|  2|   3|             5|        |      LAST CHRISTMAS|                WHAM|                 RCA|        2|            64|December 27 2019|
|  3|   4|            14|        |FAIRYTALE OF NEW ...|POGUES FT KIRSTY ...|         WARNER BROS|        2|            99|December 27 2019|
|  4|   5|          

In [8]:
df = df.drop('_c0')
df = df.drop('hmm')

In [9]:
for col in df.columns:
    df = df.withColumn(col, f.lower(f.col(col)))

In [10]:
df = df.withColumn("rank", df["rank"].cast(IntegerType()))
df = df.withColumn("peak_rank", df["peak_rank"].cast(IntegerType()))
df = df.withColumn("weeks_on_chart", df["weeks_on_chart"].cast(IntegerType()))
#df = df.withColumn("week_of", df["week_of"].cast(DateType()))

In [11]:
def to_date(x):
    return datetime.strptime(x, '%B %d %Y')
hmm = f.udf(lambda y: to_date(y), DateType())


In [12]:
df = df.withColumn("week_of", hmm('week_of'))

In [13]:
df.printSchema()

root
 |-- rank: integer (nullable = true)
 |-- last_week_rank: string (nullable = true)
 |-- title: string (nullable = true)
 |-- artist: string (nullable = true)
 |-- label: string (nullable = true)
 |-- peak_rank: integer (nullable = true)
 |-- weeks_on_chart: integer (nullable = true)
 |-- week_of: date (nullable = true)



In [14]:
df.createOrReplaceTempView('test')

In [15]:
result = spark.sql('''SELECT COUNT(*)
            FROM test
            ''')
result.show()

+--------+
|count(1)|
+--------+
|   26000|
+--------+



In [16]:
result = spark.sql('''SELECT *
            FROM test
            LIMIT 30
            ''')
result.show()

+----+--------------+--------------------+--------------------+--------------------+---------+--------------+----------+
|rank|last_week_rank|               title|              artist|               label|peak_rank|weeks_on_chart|   week_of|
+----+--------------+--------------------+--------------------+--------------------+---------+--------------+----------+
|   1|            11|               river|      ellie goulding|             polydor|        1|             5|2019-12-27|
|   2|             8|all i want for ch...|        mariah carey|            columbia|        2|            99|2019-12-27|
|   3|             5|      last christmas|                wham|                 rca|        2|            64|2019-12-27|
|   4|            14|fairytale of new ...|pogues ft kirsty ...|         warner bros|        2|            99|2019-12-27|
|   5|             2|              own it|stormzy/ed sheera...|      atlantic/merky|        2|             5|2019-12-27|
|   6|            16|merry chris

In [17]:
result = spark.sql('''
            SELECT DISTINCT title, ROUND(AVG(rank), 2)
            FROM test
            WHERE title like "%christmas%"
                OR title like "%merry%"
                OR title like "%xmas%"
                OR title like "%santa%"
                OR title like "%rudolph%"
                OR title like "%reindeer%"
            GROUP BY 1
            ORDER BY 2
            ''')
result.show(50, False)

+----------------------------------------+-----------------------------------+
|title                                   |round(avg(CAST(rank AS BIGINT)), 2)|
+----------------------------------------+-----------------------------------+
|last christmas                          |25.3                               |
|all i want for christmas is you         |26.82                              |
|happy christmas (war is over)           |28.0                               |
|merry christmas everyone                |34.46                              |
|do they know it's christmas             |35.7                               |
|i wish it could be christmas everyday   |36.0                               |
|rockin' around the christmas tree       |37.05                              |
|step into christmas                     |38.84                              |
|driving home for christmas              |39.37                              |
|santa tell me                           |41.0      

In [18]:
result = spark.sql('''
            SELECT COUNT(DISTINCT title)
            FROM test
            ''')
result.show()

+---------------------+
|count(DISTINCT title)|
+---------------------+
|                 2416|
+---------------------+



In [19]:
result = spark.sql('''
            SELECT COUNT(DISTINCT title)
            FROM test
            WHERE rank == 1''')
result.show()

+---------------------+
|count(DISTINCT title)|
+---------------------+
|                   77|
+---------------------+



In [64]:
result = spark.sql('''
            SELECT artist, title
            FROM test
            GROUP BY artist, title''').show(20, False)


+------------------------------+-----------------------------+
|artist                        |title                        |
+------------------------------+-----------------------------+
|elton john                    |step into christmas          |
|clean bandit ft ellie goulding|mama                         |
|m o/chip                      |wondering                    |
|drake                         |nonstop                      |
|florence & the machine        |big god                      |
|raye/mabel/stefflon don       |cigarette                    |
|jax jones ft raye             |you don't know me            |
|ryan gosling & emma stone     |city of stars                |
|kygo ft ella henderson        |here for you                 |
|sigma ft paloma faith         |changing                     |
|adam lambert                  |ghost town                   |
|ciara                         |i bet                        |
|charli xcx ft rita ora        |doing it               

In [65]:
result = spark.sql('''
            SELECT COUNT(*)
            FROM(
                SELECT artist, title
                FROM test
                GROUP BY artist, title)
            ''').show()

+--------+
|count(1)|
+--------+
|    2605|
+--------+



In [56]:
result = spark.sql('''
            SELECT DISTINCT artist, title, COUNT(*) as num_weeks, MIN(week_of)
            FROM test
            WHERE rank == 1
            GROUP BY 1, 2
            ORDER BY 3 DESC
            ''')
result.show(20, False)

+------------------------------+----------------------+---------+------------+
|artist                        |title                 |num_weeks|min(week_of)|
+------------------------------+----------------------+---------+------------+
|drake ft wizkid & kyla        |one dance             |15       |2016-04-15  |
|ed sheeran                    |shape of you          |14       |2017-01-13  |
|luis fonsi/daddy yankee/bieber|despacito (remix)     |11       |2017-05-12  |
|tones & i                     |dance monkey          |11       |2019-10-04  |
|clean bandit                  |rockabye              |9        |2016-11-11  |
|drake                         |god's plan            |9        |2018-01-26  |
|calvin harris & dua lipa      |one kiss              |8        |2018-04-20  |
|ed sheeran & justin bieber    |i don't care          |8        |2019-05-17  |
|lewis capaldi                 |someone you loved     |7        |2019-03-01  |
|shawn mendes/camila cabello   |senorita            

In [21]:
result = spark.sql('''
            SELECT DISTINCT title, COUNT(*) as num_weeks, MIN(week_of)
            FROM test
            WHERE peak_rank == 1
            GROUP BY 1
            ORDER BY 2 DESC
            ''')
result.show()

+-------------------+---------+------------+
|              title|num_weeks|min(week_of)|
+-------------------+---------+------------+
|       shape of you|       97|  2017-01-13|
|  thinking out loud|       90|  2015-01-11|
|            perfect|       81|  2017-12-08|
|            shotgun|       74|  2018-06-29|
|        uptown funk|       72|  2015-01-11|
|  despacito (remix)|       71|  2017-05-12|
|              sorry|       65|  2015-11-20|
|          one dance|       64|  2016-04-15|
|      love yourself|       62|  2015-12-04|
|   what do you mean|       62|  2015-09-04|
|          new rules|       61|  2017-08-18|
|         these days|       59|  2015-01-11|
|               king|       59|  2015-03-08|
|            shallow|       57|  2018-10-26|
|love me like you do|       57|  2015-02-08|
|           one kiss|       52|  2018-04-20|
|           stitches|       51|  2016-01-22|
|       hold my hand|       51|  2015-03-29|
|            7 years|       49|  2016-02-12|
|         

In [22]:
result = spark.sql('''
            SELECT AVG(num_weeks)
            FROM (SELECT DISTINCT title, COUNT(*) as num_weeks, MIN(week_of)
                    FROM test
                    WHERE peak_rank == 1
                    GROUP BY 1
                    ORDER BY 2 DESC)
            ''')
result.show()

+------------------+
|    avg(num_weeks)|
+------------------+
|26.708661417322833|
+------------------+



In [23]:
result = spark.sql('''
            SELECT AVG(rank), MAX(weeks_on_chart)
            FROM test
            WHERE title = "shape of you"
            ''')
result.show()

+-----------------+-------------------+
|        avg(rank)|max(weeks_on_chart)|
+-----------------+-------------------+
|42.51546391752577|                 97|
+-----------------+-------------------+



In [24]:
result = spark.sql('''
            SELECT artist, title, COUNT(*)
            FROM test
            GROUP BY artist, title
            ORDER BY 3 DESC
            ''')
result.show()

+--------------------+--------------------+--------+
|              artist|               title|count(1)|
+--------------------+--------------------+--------+
|          ed sheeran|             perfect|     110|
|             killers|       mr brightside|     105|
|          ed sheeran|        shape of you|      97|
|          ed sheeran|   thinking out loud|      90|
|                 sia|          chandelier|      87|
|         george ezra|             shotgun|      87|
|major lazer ft mo...|             lean on|      79|
|           james bay|           let it go|      77|
|settle/greatest s...|          this is me|      75|
|          ed sheeran|          photograph|      73|
|       walk the moon|     shut up & dance|      72|
|mark ronson ft br...|         uptown funk|      72|
|luis fonsi/daddy ...|   despacito (remix)|      71|
|   justin timberlake|can't stop the fe...|      71|
|           james bay| hold back the river|      68|
|       justin bieber|               sorry|   

In [25]:
result = spark.sql('''
            SELECT artist, title, count(*)
            FROM test
            WHERE title = "perfect"
            GROUP BY artist, title
            ''')
result.show()

+-------------+-------+--------+
|       artist|  title|count(1)|
+-------------+-------+--------+
|one direction|perfect|      21|
|   ed sheeran|perfect|     110|
+-------------+-------+--------+



In [53]:
result = spark.sql('''
            SELECT artist, title, label, count(*) as count, ROUND(AVG(rank),2) AS avg_rank, MIN(week_of)
            FROM test
            GROUP BY artist, title, label
            ORDER BY count DESC
            ''')
result.show()

+--------------------+--------------------+--------------------+-----+--------+------------+
|              artist|               title|               label|count|avg_rank|min(week_of)|
+--------------------+--------------------+--------------------+-----+--------+------------+
|          ed sheeran|             perfect|              asylum|  110|   48.09|  2017-03-10|
|             killers|       mr brightside|             mercury|  105|   84.86|  2015-04-19|
|          ed sheeran|        shape of you|              asylum|   97|   42.52|  2017-01-13|
|          ed sheeran|   thinking out loud|              asylum|   90|   56.27|  2015-01-11|
|         george ezra|             shotgun|            columbia|   87|   35.99|  2018-03-30|
|                 sia|          chandelier|   monkey puzzle/rca|   87|   64.46|  2015-01-11|
|major lazer ft mo...|             lean on|       because music|   79|   45.92|  2015-03-08|
|           james bay|           let it go|              virgin|   77|

In [27]:
result = spark.sql('''
            SELECT label, COUNT(*)
            FROM test
            GROUP BY 1
            ORDER BY 2 DESC
            ''')
result.show()

+--------------------+--------+
|               label|count(1)|
+--------------------+--------+
|            atlantic|    2055|
|            columbia|    1613|
|          interscope|    1493|
|             polydor|    1220|
|                 rca|    1099|
|              virgin|     924|
|              asylum|     911|
|                 emi|     830|
|          syco music|     821|
|    republic records|     817|
|          parlophone|     791|
|         warner bros|     653|
|              island|     650|
|cash money/republ...|     650|
|             capitol|     612|
|             def jam|     519|
|                epic|     471|
|   ministry of sound|     466|
|            positiva|     413|
|          relentless|     360|
+--------------------+--------+
only showing top 20 rows



In [28]:
result = spark.sql('''
            SELECT COUNT(DISTINCT label)
            FROM test
            ''')
result.show()

+---------------------+
|count(DISTINCT label)|
+---------------------+
|                  408|
+---------------------+



In [29]:
result = spark.sql('''
            SELECT label, COUNT(DISTINCT title) AS num_songs, ROUND(AVG(rank),2) AS avg_rank
            FROM test
            GROUP BY 1
            ORDER BY 3
            ''')
result.show()

+--------------------+---------+--------+
|               label|num_songs|avg_rank|
+--------------------+---------+--------+
|        castle music|        1|    13.0|
|            emubands|        1|    15.0|
|      asylum/def jam|        1|   17.14|
|interscope/republ...|        1|   17.56|
|cactus jack/epic/...|        1|    22.6|
|     asylum/columbia|        1|    25.0|
|      emi/syco music|        1|   25.46|
|                  zy|        1|    26.5|
|       columbia/kygo|        1|   26.72|
|black butter/def jam|        2|   26.89|
|       perfect havoc|        1|   27.86|
|def jam/polydor/r...|        1|   27.88|
|             liv'n'g|        1|    28.0|
|polydor/rca/repub...|        1|   28.44|
|  def jam/parlophone|        1|   29.06|
|      merky/atlantic|        1|   29.25|
|        fuller beans|        1|    30.0|
|          dave clark|        1|    31.0|
|   james grant music|        1|    31.0|
|black butter/dave...|        1|    31.2|
+--------------------+---------+--

In [111]:
result = spark.sql('''
            SELECT label, COUNT(DISTINCT title) AS num_songs, ROUND(AVG(rank),2) AS avg_rank
            FROM test
            GROUP BY 1
            HAVING num_songs >15
            ORDER BY 3
            ''').show(50, False)


+---------------------------+---------+--------+
|label                      |num_songs|avg_rank|
+---------------------------+---------+--------+
|republic records           |59       |43.67   |
|positiva                   |22       |44.21   |
|merky                      |16       |45.04   |
|ministry of sound          |34       |46.21   |
|warner records             |16       |46.38   |
|syco music                 |80       |46.77   |
|atlantic                   |129      |47.02   |
|disruptor                  |16       |47.25   |
|republic records/xo        |33       |47.52   |
|warner bros                |60       |47.58   |
|emi                        |76       |47.93   |
|asylum                     |43       |48.22   |
|capitol                    |48       |48.49   |
|columbia                   |134      |48.64   |
|polydor                    |110      |50.31   |
|cash money/republic records|78       |50.38   |
|3 beat                     |19       |51.3    |
|parlophone         

In [119]:
result = spark.sql('''
            SELECT label, COUNT(DISTINCT title) AS num_songs, ROUND(AVG(rank),2) AS avg_rank
            FROM test
            WHERE label NOT IN (SELECT label
                                FROM test
                                WHERE label like "%/%"
                                GROUP BY label)
            GROUP BY 1
            HAVING num_songs >15
            ORDER BY 2 DESC
            ''').show(50, False)


+-----------------+---------+--------+
|label            |num_songs|avg_rank|
+-----------------+---------+--------+
|interscope       |171      |51.51   |
|rca              |136      |53.36   |
|columbia         |134      |48.64   |
|atlantic         |129      |47.02   |
|virgin           |117      |54.65   |
|polydor          |110      |50.31   |
|island           |90       |54.77   |
|syco music       |80       |46.77   |
|parlophone       |79       |51.33   |
|emi              |76       |47.93   |
|def jam          |69       |54.3    |
|warner bros      |60       |47.58   |
|republic records |59       |43.67   |
|capitol          |48       |48.49   |
|asylum           |43       |48.22   |
|epic             |42       |53.03   |
|ministry of sound|34       |46.21   |
|relentless       |26       |54.46   |
|black butter     |22       |51.74   |
|positiva         |22       |44.21   |
|3 beat           |19       |51.3    |
|xl recordings    |17       |52.31   |
|merky            |16    

In [118]:
result = spark.sql('''
            SELECT label, COUNT(DISTINCT title), ROUND(AVG(rank),2) AS avg_rank
            FROM test
            WHERE label NOT IN (SELECT label
                                FROM test
                                WHERE label like "%/%"
                                GROUP BY label)
            GROUP BY 1
            ORDER BY 2 DESC
            ''').show(50, False)


+----------------------+---------------------+--------+
|label                 |count(DISTINCT title)|avg_rank|
+----------------------+---------------------+--------+
|interscope            |171                  |51.51   |
|rca                   |136                  |53.36   |
|columbia              |134                  |48.64   |
|atlantic              |129                  |47.02   |
|virgin                |117                  |54.65   |
|polydor               |110                  |50.31   |
|island                |90                   |54.77   |
|syco music            |80                   |46.77   |
|parlophone            |79                   |51.33   |
|emi                   |76                   |47.93   |
|def jam               |69                   |54.3    |
|warner bros           |60                   |47.58   |
|republic records      |59                   |43.67   |
|capitol               |48                   |48.49   |
|asylum                |43                   |48

In [31]:
result = spark.sql('''
            SELECT MAX(week_of), MIN(week_of)
            FROM test
            ''').show()


+------------+------------+
|max(week_of)|min(week_of)|
+------------+------------+
|  2019-12-27|  2015-01-11|
+------------+------------+



In [32]:
result = spark.sql('''
            SELECT DISTINCT artist, COUNT(*), ROUND(AVG(rank),2) AS avg_rank
            FROM test
            GROUP BY 1
            ORDER BY 2 DESC''').show()

+--------------+--------+--------+
|        artist|count(1)|avg_rank|
+--------------+--------+--------+
|    ed sheeran|     706|   49.83|
|         drake|     352|    49.4|
| justin bieber|     280|   47.48|
| ariana grande|     269|   39.33|
|   george ezra|     263|   48.57|
|    little mix|     244|   45.81|
|           sia|     244|   55.59|
|        weeknd|     228|   46.48|
|     sam smith|     216|   48.84|
|  shawn mendes|     213|   42.92|
|   jess glynne|     207|   44.33|
| years & years|     197|   50.62|
|  taylor swift|     189|   54.21|
|      dua lipa|     172|   41.03|
|     james bay|     169|   58.82|
| lewis capaldi|     158|   29.18|
|      coldplay|     157|   53.01|
|  zara larsson|     142|   43.61|
|       stormzy|     142|    49.3|
|ellie goulding|     142|   45.52|
+--------------+--------+--------+
only showing top 20 rows



In [50]:
result = spark.sql('''
            SELECT title, COUNT(*), MIN(rank) as peak, MIN(week_of)
            FROM test
            WHERE artist = "ed sheeran"
            GROUP BY 1
            ORDER BY 2 DESC
            ''').show(30, False)

+------------------------------+--------+----+------------+
|title                         |count(1)|peak|min(week_of)|
+------------------------------+--------+----+------------+
|perfect                       |110     |1   |2017-03-10  |
|shape of you                  |97      |1   |2017-01-13  |
|thinking out loud             |90      |3   |2015-01-11  |
|photograph                    |73      |15  |2015-01-11  |
|castle on the hill            |53      |2   |2017-01-13  |
|bloodstream                   |38      |2   |2015-03-01  |
|galway girl                   |31      |2   |2017-03-10  |
|sing                          |23      |29  |2015-01-11  |
|supermarket flowers           |21      |8   |2017-03-10  |
|don't                         |20      |20  |2015-01-11  |
|happier                       |18      |6   |2017-03-10  |
|what do i know                |17      |9   |2017-03-10  |
|i see fire                    |15      |54  |2015-01-11  |
|new man                       |14      

In [52]:
result = spark.sql('''
            SELECT title, COUNT(*), MIN(rank) as peak, MIN(week_of)
            FROM test
            WHERE artist = "ed sheeran"
            GROUP BY 1
            HAVING peak < 6
            ORDER BY 2 DESC
            ''').show(20, False)

+--------------------------+--------+----+------------+
|title                     |count(1)|peak|min(week_of)|
+--------------------------+--------+----+------------+
|perfect                   |110     |1   |2017-03-10  |
|shape of you              |97      |1   |2017-01-13  |
|thinking out loud         |90      |3   |2015-01-11  |
|castle on the hill        |53      |2   |2017-01-13  |
|bloodstream               |38      |2   |2015-03-01  |
|galway girl               |31      |2   |2017-03-10  |
|new man                   |14      |5   |2017-03-10  |
|how would you feel (paean)|12      |2   |2017-02-24  |
+--------------------------+--------+----+------------+



In [35]:
result = spark.sql('''
            SELECT COUNT(DISTINCT title)
            FROM test
            WHERE artist = "ed sheeran"
            
            ''').show()

+---------------------+
|count(DISTINCT title)|
+---------------------+
|                   26|
+---------------------+



In [66]:
df.printSchema()

root
 |-- rank: integer (nullable = true)
 |-- last_week_rank: string (nullable = true)
 |-- title: string (nullable = true)
 |-- artist: string (nullable = true)
 |-- label: string (nullable = true)
 |-- peak_rank: integer (nullable = true)
 |-- weeks_on_chart: integer (nullable = true)
 |-- week_of: date (nullable = true)



In [70]:
result = spark.sql('''
            SELECT DISTINCT last_week_rank
            FROM test
            ORDER BY 1 DESC''').show()

+---------------+
| last_week_rank|
+---------------+
|             re|
|            new|
|             99|
|             98|
|             97|
|             96|
|             95|
|             94|
|             93|
|             92|
|             91|
|             90|
|              9|
|             89|
|             88|
|             87|
|             86|
|             85|
|             84|
|             83|
+---------------+
only showing top 20 rows



In [85]:
result = spark.sql('''
            SELECT rank, title, artist, COUNT(*)
            FROM test
            GROUP BY 
            ''').show()

+----+--------------------+--------------------+
|rank|               title|              artist|
+----+--------------------+--------------------+
|  70|    man with the bag|            jessie j|
|  73|all i want (for c...|          liam payne|
|  86|thank god it's ch...|               queen|
|  94|a spaceman came t...|      chris de burgh|
|   1|i love sausage rolls|             ladbaby|
|   6|            audacity|stormzy ft headie...|
|   9|             lessons|             stormzy|
|  39|             falling|        harry styles|
|  48|   running the world|       jarvis cocker|
|  57| futsal shuffle 2020|        lil uzi vert|
|  77|            the gift|             sidemen|
|  83| like it's christmas|      jonas brothers|
|  94|             darling|      d-block europe|
|  98|             legends|          juice wrld|
|  99|             the box|         roddy ricch|
|  11|           adore you|        harry styles|
|  41|happy christmas (...|         john legend|
|  60|       million

In [95]:
result = spark.sql('''
            SELECT COUNT(*)
            FROM test
            WHERE weeks_on_chart = 1
            ''').show()

+--------+
|count(1)|
+--------+
|    2340|
+--------+



In [96]:
result = spark.sql('''
            SELECT COUNT(*)
            FROM test
            WHERE last_week_rank LIKE "%new%"
            ''').show()

+--------+
|count(1)|
+--------+
|    2340|
+--------+

